In [1]:
import pandas as pd 
import numpy  as np

data1 = {'user_id'   : [1,2,3,4,5,6,7],
         'name'      : ['John','Jane','Jill','Josh','Jean','Justin','Jeremy'],
         'join_date' : ['14-feb-20',
                        '14-feb-20',
                        '15-feb-20',
                        '15-feb-20',
                        '16-feb-20',
                        '17-feb-20',
                        '18-feb-20'] 
        }

data2 = {'user_id'     : [1,2,2,3,4,1,3],
         'Type'        : ['F1','F2','P','F2','F2','P','P'],
         'access_date' : ['1-mar-20',
                          '2-mar-20',
                          '12-mar-20',
                          '15-mar-20',
                          '15-mar-20',
                          '16-mar-20',
                          '22-mar-20']
        }

users  = pd.DataFrame(data1)
events = pd.DataFrame(data2)

In [2]:
users['join_date']    = pd.to_datetime(users['join_date'])
events['access_date'] = pd.to_datetime(events['access_date'])

In [3]:
users

,user_id,name,join_date
0,1,John,2020-02-14
1,2,Jane,2020-02-14
2,3,Jill,2020-02-15
3,4,Josh,2020-02-15
4,5,Jean,2020-02-16
5,6,Justin,2020-02-17
6,7,Jeremy,2020-02-18


In [6]:
events

,user_id,Type,access_date
0,1,F1,2020-03-01
1,2,F2,2020-03-02
2,2,P,2020-03-12
3,3,F2,2020-03-15
4,4,F2,2020-03-15
5,1,P,2020-03-16
6,3,P,2020-03-22


In [7]:
events['rank'] = events.groupby('user_id')['access_date'].rank(method = 'first')
f1 = np.where((events['rank'] == 1.0) & (events['Type'] == 'F2'))
F2_first = pd.DataFrame(events.loc[f1])
F2_first

,user_id,Type,access_date,rank
1,2,F2,2020-03-02,1.0
3,3,F2,2020-03-15,1.0
4,4,F2,2020-03-15,1.0


In [8]:
Premium_access = events[events.Type == 'P'][['user_id','access_date']].copy()
Premium_access.rename(columns = {'access_date':'Premium_access_date'}, inplace = True)
Premium_access 

,user_id,Premium_access_date
2,2,2020-03-12
5,1,2020-03-16
6,3,2020-03-22


In [9]:
F2_first = pd.merge(F2_first, Premium_access, on = 'user_id', how = 'left')
F2_first = pd.merge(F2_first, users, on = 'user_id', how = 'left')
F2_first 

,user_id,Type,access_date,rank,Premium_access_date,name,join_date
0,2,F2,2020-03-02,1.0,2020-03-12,Jane,2020-02-14
1,3,F2,2020-03-15,1.0,2020-03-22,Jill,2020-02-15
2,4,F2,2020-03-15,1.0,NaT,Josh,2020-02-15


In [12]:
i = (np.where(F2_first['Premium_access_date'] - F2_first['join_date'] <= pd.to_timedelta('30 days'), 1, 0).sum() / len(F2_first.index)).round(2)
d = {'upgrade_rate' : [i]
    }
upgrade = pd.DataFrame(data = d, index = [0])
upgrade

,upgrade_rate
0,0.33
